<a href="https://colab.research.google.com/github/TamizharasanG/Guvi_Tasks/blob/main/Zomato_dataset_visuvalization_and_EDA_using_plotly_and_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Packages instalation**

In [ ]:
!pip install countryinfo
!pip install easy-exchange-rates 
!pip install geopy  
!pip install pycountry
!pip install jupyter_dash
#!pip install dash
!pip install dash_bootstrap_templates

In [ ]:
import pandas as pd                           #----> used for data exploration and manipulation
import plotly.express as px                   #----> used for creating the intractive charts
import seaborn as sns                         #----> used for creating charts 
from countryinfo import CountryInfo           #----> used to find the details of the country, state , city and whole address with latitude and longitude
import numpy as np                            #----> data manipulation tool
import  datetime                              #----> used to fetch the current date and also used in keeping the time intervals
import json                                   #----> used to load and dump the json to other file formats
import matplotlib.pyplot as plt               #----> used for basic visuvalization
from plotly.subplots import make_subplots     #----> used in making subplots in plotly
import plotly.graph_objects as go             #----> used for building the more customized and intractive dashboards.


# **Part 1 : With full explination of each steps**
---

***Data Exploration***

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/nethajinirmal13/Training-datasets/main/zomato/zomato.csv")

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
sns.heatmap(data.isnull())

In [ ]:
data.isnull().sum()

In [ ]:
data.columns

In [ ]:
data["Country Code"].unique()

 TASK 1: DATA ENGINEERING

---


 **1. Add a column with rupees as the currency **

---




**Adding country name to each rows with respect to the country code**

---



In [ ]:
country_name = {1 : "India",
                14 : "Australia",
                30 : "Brazil",
                37 : "Canada",
                94 : "Indonesia",
                148 : "New Zealand",
                162 : "Phillipines",
                166 : "Qatar",
                184 : "Singapore",
                189 : "South Africa",
                191 : "Sri Lanka",
                208 : "Turkey",
                214 : "UAE",
                215 : "United Kingdom",
                216 : "United States"}

In [ ]:
Country_Name= data["Country Code"].map(country_name)

In [ ]:
data.head()

In [ ]:
# Adding the country name with respect to the country code
data.insert(3, "Country Name",Country_Name )

In [ ]:
data.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'Country Name',
       'City', 'Address', 'Locality', 'Locality Verbose', 'Longitude',
       'Latitude', 'Cuisines', 'Average Cost for two', 'Currency',
       'Has Table booking', 'Has Online delivery', 'Is delivering now',
       'Switch to order menu', 'Price range', 'Aggregate rating',
       'Rating color', 'Rating text', 'Votes'],
      dtype='object')

****FINDING THE CURRENCY CODE FOR EACH COUNTRY****

---

PACKAGE USED : countryinfo


In [ ]:
name = ['Brazil','United States', 'Australia', 'Canada', 'Singapore',"South Africa", 'UAE', 'India', 'Indonesia', 'New Zealand', 'United Kingdom','Qatar', 'Sri Lanka', 'Turkey']
a = []
for i in name:
  currencies = CountryInfo(i).currencies()
  a.append(currencies[0])

In [ ]:
a.insert(0, "PHP")

*** FINDING THE INDIAN RUPEE WITH RESPECT TO THE FOREIGN COUNTRY CURRENCY***

---

PACKAGE USED : easy-exchange-rates

In [ ]:
from easy_exchange_rates import API
api = API()
df1 = pd.DataFrame([])
ind_val = {}
x = datetime.date.today()
y = str(x)
for i in a:
  time_series = api.get_exchange_rates(
    base_currency=i,
    start_date =y ,
    end_date = y, 
    targets=["INR"]
  )
  
  
  df = api.to_dataframe(time_series)
  df1 = pd.concat([df1, df])

In [ ]:
df1["Currency code"] = a
country_name = [ "Phillipines",
                'Brazil','United States',
                'Australia',
                'Canada',
                'Singapore',
                "South Africa",
                'UAE',
                'India',
                'Indonesia',
                'New Zealand',
                'United Kingdom',
                'Qatar',
                'Sri Lanka',
                'Turkey']
df1["Country name"] = country_name
df1.columns = ["INR value", "Currency code", "Country name"]
df1 = df1.sort_values(by = "INR value", ascending = True)

**Adding column with equalent indian rupee in all datas**

---



In [ ]:
x = np.array(df1["Country name"]).reshape(-1, 1).flatten()# - Counntry name
y = np.array(df1["INR value"]).reshape(-1, 1).flatten()# - Currency value

In [ ]:
xy_zip = zip(x, y)
inr_val = {}
for i in xy_zip:
  inr_val[i[0]] = i[1]

In [ ]:
inr_value = data["Country Name"].map(inr_val)
data.insert(13, "INR Value for currency", inr_value)
avg_c_inr = data["Average Cost for two"] * data["INR Value for currency"]
data.insert(14, "Average cost  in INR", avg_c_inr)
#Rounding up to 2 decimal
data["Average cost  in INR"] = data["Average cost  in INR"].apply(lambda x:round(x, 2))

 **TASK 1: DATA ENGINEERING**


---


 **2 .Bring out a plot that compares indian currency with other country’s currency**




In [ ]:
fig = px.funnel(df1, x='INR value', y='Country name', title = "INDIAN CURRENCY CUTRRENT SITUATION( FOREIGN EXCHANGE RATES )", color = "Country name")
fig.update_layout(
  plot_bgcolor="#8EE9E7",
  paper_bgcolor="#D2C1D4",
  font_size=15,
  font_color = "#F70909"
)
fig.show()

**TASK 2 : DASHBOARD DEVELOPMENT **
----
 **● Comparison between the cities in India(own report)**

 **○ Which part of India spends more on online deliver**

 ** ○ Which part of India spends more on dine-in**

** ○ Which part has a high living cost vs low living cost**

**Using ICICLE chart**

In [ ]:
#function for finding the state name of the city
from geopy.geocoders import Nominatim
def read_country(city):
    """
    Convert cities and returns the full address with country as a tuple
    """
    geolocator = Nominatim(user_agent="google") #user agent can be any user agent 
    location = geolocator.geocode(city, language="en") #specified the language as some countries are in other lanaguages
    country= location.address.split(",") #split the string based on comma and retruns the second last element (state)
    return country

**Details of Dine In in India**

In [ ]:
#Adding column Mode of Order in the data
data["Mode of Order"] = data["Has Table booking"].map({"Yes" : "Dine In", "No" : "Online Delivery"})
Indian_data = data[data["Country Name"] == "India"]

Indian_dine_in = Indian_data[Indian_data["Mode of Order"] == "Dine In"].groupby("City").sum()
Indian_dine_in = Indian_dine_in.iloc[:, [-4, 1]]
#Indian_dine_in
Indian_dine_in.columns = ["Total Cost in INR", "No of Orders"]
Indian_dine_in["Mode of Order"] = "Dine In"
Indian_dine_in["City"] = Indian_dine_in.index
Indian_dine_in["S No"] = [i for i in range(1, Indian_dine_in.shape[0]+1)]
Indian_dine_in.set_index("S No", inplace = True)

In [ ]:
States_d = []
for i in Indian_dine_in["City"]:
    x = read_country(i)
    y = x[-2].strip(" ")
    if y.isdigit():
        States_d.append(x[-3].strip(" "))
    elif not(y.isdigit()):
        States_d.append(x[-2].strip(" "))

Indian_dine_in["State"] = States_d
#log is taken for the better visuvalization
Indian_dine_in["Total cost in log"] = Indian_dine_in['Total Cost in INR'].apply(lambda x: np.log(x))

**Details of Online orders by cities in Indiia**

In [ ]:
Indian_online_order = Indian_data[Indian_data["Mode of Order"] == "Online Delivery"].groupby("City").sum()
Indian_online_order = Indian_online_order.iloc[:, [-4, 1]]
Indian_online_order.columns = ["Total Cost in INR", "No of Orders"]
Indian_online_order["Mode of Order"] = "Online Delivery"
Indian_online_order["City"] = Indian_online_order.index
Indian_online_order["S No"] = [i for i in range(1, Indian_online_order.shape[0]+1)]
Indian_online_order.set_index("S No", inplace = True)
Indian_online_order.columns

Index(['Total Cost in INR', 'No of Orders', 'Mode of Order', 'City'], dtype='object')

In [ ]:
States_o = []
for i in Indian_online_order["City"]:
    x = read_country(i)
    y = x[-2].strip(" ")
    if y.isdigit():
        States_o.append(x[-3].strip(" "))
    elif not(y.isdigit()):
        States_o.append(x[-2].strip(" "))

# Gujarat state names mising in the package
States_o[-4] = "Gujarat"
Indian_online_order["State"] = States_o
Indian_online_order["Total cost in log"] = Indian_online_order["Total Cost in INR"].apply(lambda x: np.log(x))

**Overall details of the India Cities**

In [ ]:
overall = [Indian_dine_in, Indian_online_order]
overall_indian_data = pd.concat(overall)
overall_indian_data["S.No"] = [i for i in range(1, overall_indian_data.shape[0]+1)]
overall_indian_data.set_index("S.No", inplace = True)

**Comparison of Dine In gross revanue between Indian cities**

In [ ]:
fig1 = px.icicle(Indian_dine_in, path=[px.Constant("all"), 'Mode of Order','City', "State"],
                values='Total cost in log', color='State', hover_name = "Total Cost in INR", hover_data = ["No of Orders", "Total Cost in INR"])
fig1.update_layout(margin = dict(t=50, l=25, r=25, b=25))

**Comparison of Online order gross revanue between Indian cities**

In [ ]:
fig2 = px.icicle(Indian_online_order, path=[px.Constant("all"), 'Mode of Order','City',  "State"],
                values='Total cost in log', color='State', hover_name = "Total Cost in INR", hover_data = ["No of Orders", "Total Cost in INR"])
fig2.update_layout(margin = dict(t=50, l=25, r=25, b=25))

**Overall comparision of Indian cities revanue based on Dine IN and Online delivery**

In [ ]:
fig6 = px.icicle(overall_indian_data, path=[px.Constant("all"), 'Mode of Order','City', "State"],
                values='Total cost in log', color='State', hover_name = "Total Cost in INR", hover_data = ["No of Orders", "Total Cost in INR"])
fig6.update_layout(margin = dict(t=50, l=25, r=25, b=25))

 **Finding the famous cuisiens of each city in india**

In [ ]:
city = Indian_data["City"].unique()

In [ ]:
#function for finding the city wise fav food
#counter is imported here to find the favourite cusien of the city
from collections import Counter
def fav_cuisien(y):
    from heapq import nlargest
    c1 = []
    c2 = []
    for i in Indian_data[Indian_data["City"] == y]["Cuisines"].values:
        c1.append(i.split(","))
    for i in c1:
        for j in i:
            c2.append(j)
    l = dict(Counter(c2))
    return nlargest(1, l, key = l.get)[0]

In [ ]:
food = []
for i in city:
    food.append(fav_cuisien(i))

In [ ]:
value = {"City" : city, "Favorite Cuisines" : food}
f_food_of_cities = pd.DataFrame(value)

In [ ]:
f_food_of_cities


In [ ]:
'''
Loop for finding the coustliest dishes of the cities
'''
coustlu_cuisines = []
for i in city:
    coustlu_cuisines.append(Indian_data[Indian_data["City"] == i].sort_values("Average cost  in INR", ascending = False)["Cuisines"].head(1).values)
    

costly_cuisiens = []
for i in range(len(coustlu_cuisines)):
    costly_cuisiens.append(coustlu_cuisines[i][0]) 

In [ ]:
f_food_of_cities["Costly Cuisines"] = costly_cuisiens

**City wise analysis(INDIA) only**


In [ ]:
#Functionss for creating informations for the tabel
class City_data:
    def __init__(self, city_name):
        self.city_name = city_name
    
    #Function to find the favorite cuisiens of the c
    def top5_cuisiens_indian_cities(self):
        c1 = []
        c2 = []
        for i in Indian_data[Indian_data["City"] == self.city_name]["Cuisines"].values:
            c1.append(i.split(","))
        for i in c1:
            for j in i:
                c2.append(j)
        l = dict(Counter(c2))
        from heapq import nlargest
        res = nlargest(5, l, key = l.get)
        return res

    #Function to find the costly cuisien combinations of the people of a specific city
    def top5_costly_cusiens_combinations_of_indian_cities(self):
        return Indian_data[Indian_data["City"] == self.city_name].sort_values("Average cost  in INR", ascending = False)["Cuisines"].head().tolist()
    
    #Function to find the top frequent restraunts in the city
    def top_picks_of_people(self):
        return Indian_data[Indian_data["City"] == self.city_name]["Restaurant Name"].value_counts().head(5).index
    
    #Function to find the top 5 high revaneu restaurant of the city
    def high_revaneu_restaurants(self):
        return Indian_data[Indian_data["City"] == self.city_name].groupby("Restaurant Name").sum().sort_values("Average cost  in INR", ascending = False).head().index
    
    #Function to find the least revaneu restarunts of the city
    def low_revaneu_restarunts(self):
        return Indian_data[Indian_data["City"] == self.city_name].sort_values("Average cost  in INR", ascending = True)["Restaurant Name"].head().tolist()

#********************************************************************************************************************************************************************************************
#Function for tabulating city data
def tabulate_city_data(x):
    nx = City_data(x)
    cusiens = nx.top5_cuisiens_indian_cities()
    combination = nx.top5_costly_cusiens_combinations_of_indian_cities()
    top_picks = nx.top_picks_of_people()
    high_revaneu = nx.high_revaneu_restaurants()
    low_revaneu = nx.low_revaneu_restarunts()

    c_data=  pd.DataFrame({"Rank" : [1, 2, 3, 4, 5],"Favourite cuisiens" : cusiens,
                              "Top 5 costly cuisines combination" : combination,
                             "Top picked Restarunts" : top_picks,
                             "High revaneu restarunts" : high_revaneu,
                             "Low revaneu restarunts" : low_revaneu})
    fig = go.Figure(data=[go.Table(
        columnorder = [1,2, 3, 4, 5, 6],
        columnwidth = [50,100, 250, 150, 150, 150],
        header = dict(
            values = [['<b>'+i+'<b>'] for i in c_data.columns],
            line_color='red',
            fill_color='black',
            align=['center'],
            font=dict(color='red', size=15),
            height=40
      ),
      cells=dict(
        values=[c_data.Rank,
                c_data["Favourite cuisiens"], 
                c_data["Top 5 costly cuisines combination"], 
                c_data["Top picked Restarunts"], 
                c_data["High revaneu restarunts"],
                c_data["Low revaneu restarunts"]
               ],
        line_color='red',
        fill=dict(color=['white', 'white']),
        fill_color='white',
        align=['center'],
        font_size=15,
        height=30)
        )
    ])
    fig.update_layout(height = 300,width = 1300,
                      paper_bgcolor = "#091335",
                       autosize=False,
                       margin = dict(
                           l=20,
                           r=20,
                           b=0,
                           t=10,
                           pad=4,
                           autoexpand=True))
    return fig
    
#********************************************************************************************************************************************************************************************
#Function for creating bar and pie chart of country wise analysis
def city_plot(cty_name):
  fig = make_subplots(
      rows=1, cols=2,
      subplot_titles=(f'Spending habit of people in the {cty_name}',
                      'Dine IN vs Online delivery'),
      specs=[[{"type": "xy"}, {"type": "domain"}]],         
  )

  fig.add_trace(
      go.Box(x = Indian_data[Indian_data["City"] == cty_name]["Average cost  in INR"], showlegend=False), row = 1, col = 1)
  
  fig.add_trace(go.Pie(values = Indian_data[Indian_data["City"] == cty_name]["Mode of Order"].value_counts().tolist(),
                        labels = Indian_data[Indian_data["City"] == cty_name]["Mode of Order"].value_counts().index,
                        textinfo="label+percent",
                        showlegend=False),
                  row=1, col=2)

  fig.update_layout(height=400,
                    width = 1500,
                     autosize=False,
                    margin = dict(
                        l=30,
                        r=0,
                        b=30,
                        t=50,
                        pad=4,
                        autoexpand=False),
                    template = "plotly_dark",
                    plot_bgcolor = "#A3ACAC",
                    paper_bgcolor = "#091335 ",
                    showlegend=False)
  return fig


**Country wise data analysis**

In [ ]:
#Counter will not work woth nan vaues so removing the nan values.
data.dropna(axis = 0, inplace = True)

#Function to find the fav cuisiens of countries
def fav_cuisiens_countries(x):
    c1 = []
    c2 = []
    
    for i in data[data["Country Name"] == x]["Cuisines"].values:
        c1.append(i.split(","))
    for i in c1:
        for j in i:
            c2.append(j)
    l = dict(Counter(c2))
    from heapq import nlargest
    res = nlargest(1, l, key = l.get)
    return res
#top revanued city
def max_perticipant(x):
    max_city = data[data["Country Name"] == x].groupby("City").sum().sort_values("Average cost  in INR", ascending = False).index[0]
    return max_city
#least revanued city
def min_perticipant(x):
    min_city = data[data["Country Name"] == x].groupby("City").sum().sort_values("Average cost  in INR", ascending = False).index[-1]
    return min_city
#top revaneu in a single city
def amt_max_cont(x):
    amt_max = data[data["Country Name"] == x].groupby("City").sum().sort_values("Average cost  in INR", ascending = False).iloc[0, 6]
    return amt_max
#leadt revanue in a single city
def amt_min_cont(x):
    amt_min = data[data["Country Name"] == x].groupby("City").sum().sort_values("Average cost  in INR", ascending = False).iloc[-1, 6]
    return amt_min

def iso_code(x):    
    import pycountry
    user_input = x
    mapping = {country.name: country.alpha_3 for country in pycountry.countries}
    return mapping.get(user_input)

In [ ]:
Country_data = data.groupby("Country Name").mean().iloc[:, [6,7,8]]
Country_data["Total amount of money spent in a country"] = data.groupby("Country Name").sum().iloc[:, 6]
Country_data["Sum of voted in a country"] = data.groupby("Country Name").sum().iloc[:, 9]
Country_data["Country Name"] = Country_data.index
Country_data["S.No"] = [i for i in range(1, Country_data.shape[0]+1)]
Country_data.set_index("S.No", inplace = True)
Country_data["Maximum contributed city"] = Country_data["Country Name"].apply(lambda x:max_perticipant(x))
Country_data["Maximum contributed amount"] = Country_data["Country Name"].apply(lambda x:amt_max_cont(x))
Country_data["Minimum contributed city"] = Country_data["Country Name"].apply(lambda x:min_perticipant(x))
Country_data["Minimum contributed amount"] = Country_data["Country Name"].apply(lambda x:amt_min_cont(x))
Country_data["Favourite cuisien"] = Country_data["Country Name"].apply(lambda x:fav_cuisiens_countries(x)[0])
Country_data["ISO_alpha"] = Country_data["Country Name"].apply(lambda x:iso_code(x))
Country_data.iloc[-3, -1] = "UAE"
Country_data.iloc[6, -1] = "PHL"
online_order_by_country = {}
dine_in_by_country = {}
for i in zip(data[data["Mode of Order"] == "Dine In"].groupby("Country Name").sum().iloc[:, 6].index, data[data["Mode of Order"] == "Dine In"].groupby("Country Name").sum().iloc[:, 6].tolist()):
  dine_in_by_country[i[0]] = round(i[1], 2)
for i in zip(data[data["Mode of Order"] == "Online Delivery"].groupby("Country Name").sum().iloc[:, 6].index, data[data["Mode of Order"] == "Online Delivery"].groupby("Country Name").sum().iloc[:, 6].tolist()):
  online_order_by_country[i[0]] = round(i[1], 2)

Country_data["Revanue by Online delivery"] = Country_data["Country Name"].map(online_order_by_country)
Country_data["Revanue by Dine In"] = Country_data["Country Name"].map(dine_in_by_country)
#Since dine in data of some countries were not available it is filled zero for further manipulation
Country_data.fillna(0, inplace = True)

In [ ]:
#Class of functiond for creating the content for the table
class Country_details:
  def __init__(self, c_name):
    self.c_name = c_name

  def fav_cuis_country(self):
    cuis = Country_data[Country_data["Country Name"] == self.c_name].iloc[0, -4]
    return cuis

  def t_contribution(self):
    t_cont = Country_data[Country_data["Country Name"] == self.c_name].iloc[0, 3]
    return "₹"+str(round(t_cont, 2))

  def avg_spend(self):
    avg_spnd = Country_data[Country_data["Country Name"] == self.c_name].iloc[0, 0]
    return "₹"+str(round(avg_spnd, 2))

  def max_city(self):
    max_city = Country_data[Country_data["Country Name"] == self.c_name].iloc[0, 6]
    return max_city
  
  def min_city(self):
    min_city = Country_data[Country_data["Country Name"] == self.c_name].iloc[0, 8]
    return min_city

#*********************************************************************************************************************************************************************************
#Function for creating table for the country wise analysis
def tabulate_country_data(x):
    country_analysis = Country_details(x)
    o1 = country_analysis.fav_cuis_country()
    o2 = country_analysis.t_contribution()
    o3 = country_analysis.avg_spend()
    o4 = country_analysis.max_city()
    o5 = country_analysis.min_city()
    ctry_data=  pd.DataFrame({"Favourite Cuisine" : o1,
                              "Total contribution" : o2,
                              "Average spending ability of people" : o3,
                              "Maximum contributed city" : o4,
                              "Minimum contributed city" : o5}, index = [1])
    fig = go.Figure(data=[go.Table(
        columnorder = [1,2, 3, 4, 5],
        columnwidth = [100,100, 150, 150, 150],
        header = dict(
            values = [['<b>'+i+'<b>'] for i in ctry_data.columns],
            line_color='red',
            fill_color='black',
            align=['center'],
            font=dict(color='red', size=15),
            height=40
      ),
      cells=dict(
        values=[ctry_data["Favourite Cuisine"],
                ctry_data["Total contribution"], 
                ctry_data["Average spending ability of people"], 
                ctry_data["Maximum contributed city"], 
                ctry_data["Minimum contributed city"]
               ],
        line_color='red',
        fill=dict(color=['white', 'white']),
        align=['center'],
        font_size=15,
        height=30)
        )
    ])
    fig.update_layout(width=1500, height=290, paper_bgcolor = "#091335", title = "Tabular representation of country")
    
    return fig

#***************************************************************************************************************************************************************************
#Function for creating bar and pie chart of country wise analysis
def country_plot(c_name):
  fig = make_subplots(
      rows=1, cols=2,
      subplot_titles=(f'Spending habit of people in the country {c_name} in INR',
                      'Dine IN vs Online delivery'),
      specs=[[{"type": "xy"}, {"type": "domain"}]],         
  )

  fig.add_trace(
      go.Box(x = data[data["Country Name"] == c_name]["Average cost  in INR"], showlegend=False), row = 1, col = 1)
  
  fig.add_trace(go.Pie(values = data[data["Country Name"] == c_name]["Mode of Order"].value_counts().tolist(),
                        labels = data[data["Country Name"] == c_name]["Mode of Order"].value_counts().index,
                        textinfo="label+percent",
                        showlegend=False),
                  row=1, col=2)

  fig.update_layout(height=500,
                    width = 1500,
                    template = "plotly_dark",
                    plot_bgcolor = "#A3ACAC",
                    paper_bgcolor = "#091335",
                    showlegend=False)
  return fig


**World wide analysis**

In [ ]:
def choropleth_map(feature):
  money = ["Average cost  in INR", "Total amount of money spent in a country", "Revanue by Online delivery", "Revanue by Dine In"]
  if feature in money:
    ct = '₹'
  elif feature not in money:
    ct = ""

  fig = go.Figure(data=go.Choropleth(
      locations = Country_data['ISO_alpha'],
      z = Country_data[feature],
      text = Country_data['Country Name'],
      autocolorscale=True,
      reversescale=True,
      marker_line_color='black',
      marker_line_width=0.5,
      colorbar_tickprefix = ct,
      colorbar_title = feature
  ))

  fig.update_layout(
      autosize=False,
      margin = dict(
                l=2,
                r=0,
                b=0,
                t=0,
                pad=4,
                autoexpand=True
            ),height = 500, width = 1500,
            paper_bgcolor = "#091335",
      geo=dict(
          showframe=False,
          showcoastlines=False,
          projection_type='equirectangular',
          showocean=True, oceancolor="LightBlue"
      ),
      annotations = [dict(
          x=0.1,
          y=0.1,
          xref='paper',
          yref='paper',
          showarrow = False
      )]
  )
  return fig

def scatter_map(feature):
  money = ["Average cost  in INR", "Total amount of money spent in a country", "Revanue by Online delivery", "Revanue by Dine In"]
  if feature in money:
    ct = '₹'
  elif feature not in money:
    ct = ""
  fig = px.scatter_geo(Country_data, locations="ISO_alpha",
                      color=feature, # lifeExp is a column of gapminder
                      hover_name="Country Name", # column to add to hover information
                      color_continuous_scale=px.colors.sequential.Plasma,
                      size = feature,
                      hover_data = ["Favourite cuisien",
                                    "Maximum contributed city", 
                                    "Maximum contributed amount",
                                    "Minimum contributed city",
                                    "Minimum contributed amount",
                                    ])
  fig.update_layout(
      autosize=False,
      margin = dict(
                l=2,
                r=0,
                b=0,
                t=0,
                pad=4,
                autoexpand=True
            ),height = 500, width = 1500,
            paper_bgcolor = "#091335",
      geo=dict(
          showframe=False,
          showcoastlines=False,
          projection_type='equirectangular',
          showocean=True, 
          oceancolor="LightBlue"
      ))
  return fig




**Overall dashboard developement**

In [20]:
#from dash import Dash
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
#
load_figure_template(["sketchy", "cyborg", "minty"])
#app = Dash(__name__)

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.VAPOR])

fig6 = px.icicle(overall_indian_data, path=[px.Constant("all"), 'Mode of Order','City', "State"],
                values='Total cost in log', color='State', hover_name = "Total Cost in INR", hover_data = ["No of Orders", "Total Cost in INR"])
fig6.update_layout(margin = dict(t=50, l=25, r=25, b=25), paper_bgcolor = "#091335")

app.layout = html.Div([
   html.H1("Zomato Conutry wise sales analysis", style = {"color" : "red", "textAlign"  : "center"}),
   html.Br(),
   html.H4("Select the Country"),
   html.Div([
       dcc.Dropdown(
           id = "countryname",
           options = [{"label" : x, "value" : x} for x in data["Country Name"].unique()],
           placeholder = "India",
           clearable=True   
       )
   ]),
   html.Br(),
   dcc.Graph(
       id = "fig3"
   ),
   #html.Br(),
   dcc.Graph(
       id = "fig4"
       ),
     html.H1("Zomato data Indian Citywise analysis", style = {"color" : "red", "textAlign"  : "center"}),
     html.Br(),
     html.H4("Select the city"),
   html.Div([
       dcc.Dropdown(
           id = "cityname",
           options = [{"label" : x, "value" : x} for x in Indian_data["City"].unique()],
           value = "Chennai",
           clearable=True   
       )
   ]),
   html.Br(),
   dcc.Graph(
       id = "fig1"
   ),
   html.Br(),
   dcc.Graph(
       id = "fig2"
       ),
    html.Br(),
    html.H1("Revanue of the Indian Cities with respect to their delivery modes", style = {"color" : "red", "textAlign" : "center", "size" : 15}),
    dcc.Graph(
        id = "mode_of_delivery_analysis",
        figure = fig6

  ),
  html.H1("Zomato worldwide sales analysis", style = {"color" : "red", "textAlign"  : "center"}),
  html.H4("Select the featur"),
  html.Div([
      dcc.Dropdown(
          id = "featurename",
          options = [{"label" : "Country wise spending nature" ,"value" :  "Average cost  in INR"},
                     {"label" : "Gross revaneu by country" , "value" : "Total amount of money spent in a country"},
                     {"label" : "Revaneu by Online delivery" , "value" : "Revanue by Online delivery"},
                     {"label" : "Revaneu by Dine In" , "value" : "Revanue by Dine In"},
                     {"label" : "Average Rating by the customers by country" , "value" : "Aggregate rating"},
                     {"label" : "Prefered price range by country" , "value" : "Price range"}],
                   value = "Gross revaneu by country",
                   clearable=True   
       )
   ]),
   html.Br(),
   html.H4("Select the map type"),
   dcc.RadioItems(
       id = "map_selection",
       options = ['Choropleth Visuvalization','Scattered Visuvalization'],
       value =  'Choropleth Visuvalization'),
  #html.Br(),
  html.H2([], 
          style = {"Color" : "red", "textAlign" : "center", "size" : 15},
          id = "map_name"),
  html.Br(),

   dcc.Graph(
       id = "fig5"
   ),
   html.Br(),
   html.H6("This dashboard is only for educational purpose and developed by G.TAMIZHARASAN", style = {"textAlign" : "center"})

   
])
 
 
@app.callback(
    Output(component_id='fig3', component_property='figure'),
    Input(component_id='countryname', component_property='value'),
    prevent_initial_call=True
)
def update_tabel_country(input_value):
  return tabulate_country_data(input_value)

@app.callback(
    Output(component_id='fig4', component_property='figure'),
    Input(component_id='countryname', component_property='value'),
    prevent_initial_call=True
)
def update_charts_country(input_value):
  return country_plot(input_value)

@app.callback(
    Output(component_id='fig1', component_property='figure'),
    Input(component_id='cityname', component_property='value'),
    prevent_initial_call=True
)
def update_tabel_city(input_value):
  return tabulate_city_data(input_value)

@app.callback(
    Output(component_id='fig2', component_property='figure'),
    Input(component_id='cityname', component_property='value'),
    prevent_initial_call=True
)
def update_chart_india(input_value):
  return city_plot(input_value)

@app.callback(
    Output(component_id='map_name', component_property='children'),
    Input(component_id='featurename', component_property='value'),
)
def title(name): 
  return name.upper()

 
@app.callback(
    Output(component_id='fig5', component_property='figure'),
    Input(component_id='featurename', component_property='value'),
    Input(component_id='map_selection', component_property='value'),
    prevent_initial_call=True
)
def update_map(comp, map_selection):
  if map_selection == "Choropleth Visuvalization":
    return choropleth_map(comp)
  elif map_selection == "Scattered Visuvalization":
    return scatter_map(comp)




if __name__ == '__main__':
   app.run_server(debug = True, mode = "external", port = 8600)

Dash app running on:


<IPython.core.display.Javascript object>

## **Part 2 : This part contains raw code for the overall dash developement same thing in the above parts without any explinations**

In [ ]:
!pip install countryinfo
!pip install easy-exchange-rates 
!pip install geopy  
!pip install pycountry
!pip install jupyter_dash
!pip install dash
!pip install dash_bootstrap_templates
!pip install dash_bootstrap_components

import pandas as pd
import plotly.express as px
import seaborn as sns
from countryinfo import CountryInfo
import numpy as np
import  datetime
import json
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from dash_bootstrap_templates import load_figure_template

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/nethajinirmal13/Training-datasets/main/zomato/zomato.csv")

In [3]:
country_name = {1 : "India",
                14 : "Australia",
                30 : "Brazil",
                37 : "Canada",
                94 : "Indonesia",
                148 : "New Zealand",
                162 : "Phillipines",
                166 : "Qatar",
                184 : "Singapore",
                189 : "South Africa",
                191 : "Sri Lanka",
                208 : "Turkey",
                214 : "UAE",
                215 : "United Kingdom",
                216 : "United States"}

Country_Name= data["Country Code"].map(country_name)
data.insert(3, "Country Name",Country_Name )
name = ['Brazil','United States', 'Australia', 'Canada', 'Singapore',"South Africa", 'UAE', 'India', 'Indonesia', 'New Zealand', 'United Kingdom','Qatar', 'Sri Lanka', 'Turkey']
a = []
for i in name:
  currencies = CountryInfo(i).currencies()
  a.append(currencies[0])

a.insert(0, "PHP")

In [4]:
from easy_exchange_rates import API
api = API()
df1 = pd.DataFrame([])
ind_val = {}
x = datetime.date.today()
y = str(x)
for i in a:
  time_series = api.get_exchange_rates(
    base_currency=i,
    start_date =y ,
    end_date = y, 
    targets=["INR"]
  )
  
  
  df = api.to_dataframe(time_series)
  df1 = pd.concat([df1, df])

In [ ]:
df1["Currency code"] = a
country_name = [ "Phillipines",
                'Brazil','United States',
                'Australia',
                'Canada',
                'Singapore',
                "South Africa",
                'UAE',
                'India',
                'Indonesia',
                'New Zealand',
                'United Kingdom',
                'Qatar',
                'Sri Lanka',
                'Turkey']
df1["Country name"] = country_name
df1.columns = ["INR value", "Currency code", "Country name"]
df1 = df1.sort_values(by = "INR value", ascending = True)

In [6]:
x = np.array(df1["Country name"]).reshape(-1, 1).flatten()# - Counntry name
y = np.array(df1["INR value"]).reshape(-1, 1).flatten()# - Currency value

xy_zip = zip(x, y)
inr_val = {}
for i in xy_zip:
  inr_val[i[0]] = i[1]

#Adding INR value of the each country to the original dataframe
inr_value = data["Country Name"].map(inr_val)
data.insert(13, "INR Value for currency", inr_value)
avg_c_inr = data["Average Cost for two"] * data["INR Value for currency"]
data.insert(14, "Average cost  in INR", avg_c_inr)
#Rounding up to 2 decimal
data["Average cost  in INR"] = data["Average cost  in INR"].apply(lambda x:round(x, 2))

In [7]:
#function for finding the state name of the city
from geopy.geocoders import Nominatim
def read_country(city):
    """
    Convert cities and returns the full address with country as a tuple
    """
    geolocator = Nominatim(user_agent="google") #user agent can be any user agent 
    location = geolocator.geocode(city, language="en") #specified the language as some countries are in other lanaguages
    country= location.address.split(",") #split the string based on comma and retruns the second last element (state)
    return country

In [8]:
#Adding column Mode of Order in the data
data["Mode of Order"] = data["Has Table booking"].map({"Yes" : "Dine In", "No" : "Online Delivery"})
Indian_data = data[data["Country Name"] == "India"]

Indian_dine_in = Indian_data[Indian_data["Mode of Order"] == "Dine In"].groupby("City").sum()
Indian_dine_in = Indian_dine_in.iloc[:, [-4, 1]]
#Indian_dine_in
Indian_dine_in.columns = ["Total Cost in INR", "No of Orders"]
Indian_dine_in["Mode of Order"] = "Dine In"
Indian_dine_in["City"] = Indian_dine_in.index
Indian_dine_in["S No"] = [i for i in range(1, Indian_dine_in.shape[0]+1)]
Indian_dine_in.set_index("S No", inplace = True)

In [9]:
States_d = []
for i in Indian_dine_in["City"]:
    x = read_country(i)
    y = x[-2].strip(" ")
    if y.isdigit():
        States_d.append(x[-3].strip(" "))
    elif not(y.isdigit()):
        States_d.append(x[-2].strip(" "))

Indian_dine_in["State"] = States_d
#log is taken for the better visuvalization
Indian_dine_in["Total cost in log"] = Indian_dine_in['Total Cost in INR'].apply(lambda x: np.log(x))

In [10]:
Indian_online_order = Indian_data[Indian_data["Mode of Order"] == "Online Delivery"].groupby("City").sum()
Indian_online_order = Indian_online_order.iloc[:, [-4, 1]]
Indian_online_order.columns = ["Total Cost in INR", "No of Orders"]
Indian_online_order["Mode of Order"] = "Online Delivery"
Indian_online_order["City"] = Indian_online_order.index
Indian_online_order["S No"] = [i for i in range(1, Indian_online_order.shape[0]+1)]
Indian_online_order.set_index("S No", inplace = True)

In [11]:
States_o = []
for i in Indian_online_order["City"]:
    x = read_country(i)
    y = x[-2].strip(" ")
    if y.isdigit():
        States_o.append(x[-3].strip(" "))
    elif not(y.isdigit()):
        States_o.append(x[-2].strip(" "))

# Gujarat state names mising in the package
States_o[-4] = "Gujarat"
Indian_online_order["State"] = States_o
Indian_online_order["Total cost in log"] = Indian_online_order["Total Cost in INR"].apply(lambda x: np.log(x))

In [12]:
overall = [Indian_dine_in, Indian_online_order]
overall_indian_data = pd.concat(overall)
overall_indian_data["S.No"] = [i for i in range(1, overall_indian_data.shape[0]+1)]
overall_indian_data.set_index("S.No", inplace = True)

In [13]:
city = Indian_data["City"].unique()
#function for finding the city wise fav food
#counter is imported here to find the favourite cusien of the city
from collections import Counter
def fav_cuisien(y):
    from heapq import nlargest
    c1 = []
    c2 = []
    for i in Indian_data[Indian_data["City"] == y]["Cuisines"].values:
        c1.append(i.split(","))
    for i in c1:
        for j in i:
            c2.append(j)
    l = dict(Counter(c2))
    return nlargest(1, l, key = l.get)[0]

food = []
for i in city:
    food.append(fav_cuisien(i))

value = {"City" : city, "Favorite Cuisines" : food}
f_food_of_cities = pd.DataFrame(value)

'''
Loop for finding the coustliest dishes of the cities
'''
coustlu_cuisines = []
for i in city:
    coustlu_cuisines.append(Indian_data[Indian_data["City"] == i].sort_values("Average cost  in INR", ascending = False)["Cuisines"].head(1).values)
    

costly_cuisiens = []
for i in range(len(coustlu_cuisines)):
    costly_cuisiens.append(coustlu_cuisines[i][0]) 

f_food_of_cities["Costly Cuisines"] = costly_cuisiens

In [14]:
#Functionss for creating informations for the tabel
class City_data:
    def __init__(self, city_name):
        self.city_name = city_name
    
    #Function to find the favorite cuisiens of the c
    def top5_cuisiens_indian_cities(self):
        c1 = []
        c2 = []
        for i in Indian_data[Indian_data["City"] == self.city_name]["Cuisines"].values:
            c1.append(i.split(","))
        for i in c1:
            for j in i:
                c2.append(j)
        l = dict(Counter(c2))
        from heapq import nlargest
        res = nlargest(5, l, key = l.get)
        return res

    #Function to find the costly cuisien combinations of the people of a specific city
    def top5_costly_cusiens_combinations_of_indian_cities(self):
        return Indian_data[Indian_data["City"] == self.city_name].sort_values("Average cost  in INR", ascending = False)["Cuisines"].head().tolist()
    
    #Function to find the top frequent restraunts in the city
    def top_picks_of_people(self):
        return Indian_data[Indian_data["City"] == self.city_name]["Restaurant Name"].value_counts().head(5).index
    
    #Function to find the top 5 high revaneu restaurant of the city
    def high_revaneu_restaurants(self):
        return Indian_data[Indian_data["City"] == self.city_name].groupby("Restaurant Name").sum().sort_values("Average cost  in INR", ascending = False).head().index
    
    #Function to find the least revaneu restarunts of the city
    def low_revaneu_restarunts(self):
        return Indian_data[Indian_data["City"] == self.city_name].sort_values("Average cost  in INR", ascending = True)["Restaurant Name"].head().tolist()

#********************************************************************************************************************************************************************************************
#Function for tabulating city data
def tabulate_city_data(x):
    nx = City_data(x)
    cusiens = nx.top5_cuisiens_indian_cities()
    combination = nx.top5_costly_cusiens_combinations_of_indian_cities()
    top_picks = nx.top_picks_of_people()
    high_revaneu = nx.high_revaneu_restaurants()
    low_revaneu = nx.low_revaneu_restarunts()

    c_data=  pd.DataFrame({"Rank" : [1, 2, 3, 4, 5],"Favourite cuisiens" : cusiens,
                              "Top 5 costly cuisines combination" : combination,
                             "Top picked Restarunts" : top_picks,
                             "High revaneu restarunts" : high_revaneu,
                             "Low revaneu restarunts" : low_revaneu})
    fig = go.Figure(data=[go.Table(
        columnorder = [1,2, 3, 4, 5, 6],
        columnwidth = [50,100, 250, 150, 150, 150],
        header = dict(
            values = [['<b>'+i+'<b>'] for i in c_data.columns],
            line_color='red',
            fill_color='black',
            align=['center'],
            font=dict(color='red', size=15),
            height=40
      ),
      cells=dict(
        values=[c_data.Rank,
                c_data["Favourite cuisiens"], 
                c_data["Top 5 costly cuisines combination"], 
                c_data["Top picked Restarunts"], 
                c_data["High revaneu restarunts"],
                c_data["Low revaneu restarunts"]
               ],
        line_color='red',
        fill=dict(color=['white', 'white']),
        fill_color='white',
        align=['center'],
        font_size=15,
        height=30)
        )
    ])
    fig.update_layout(height = 300,width = 1300,
                      paper_bgcolor = "#091335",
                       autosize=False,
                       margin = dict(
                           l=20,
                           r=20,
                           b=0,
                           t=10,
                           pad=4,
                           autoexpand=True))
    return fig
    
#********************************************************************************************************************************************************************************************
#Function for creating bar and pie chart of country wise analysis
def city_plot(cty_name):
  fig = make_subplots(
      rows=1, cols=2,
      subplot_titles=(f'Spending habit of people in the {cty_name}',
                      'Dine IN vs Online delivery'),
      specs=[[{"type": "xy"}, {"type": "domain"}]],         
  )

  fig.add_trace(
      go.Box(x = Indian_data[Indian_data["City"] == cty_name]["Average cost  in INR"], showlegend=False), row = 1, col = 1)
  
  fig.add_trace(go.Pie(values = Indian_data[Indian_data["City"] == cty_name]["Mode of Order"].value_counts().tolist(),
                        labels = Indian_data[Indian_data["City"] == cty_name]["Mode of Order"].value_counts().index,
                        textinfo="label+percent",
                        showlegend=False),
                  row=1, col=2)

  fig.update_layout(height=400,
                    width = 1500,
                     autosize=False,
                    margin = dict(
                        l=30,
                        r=0,
                        b=30,
                        t=50,
                        pad=4,
                        autoexpand=False),
                    template = "plotly_dark",
                    plot_bgcolor = "#A3ACAC",
                    paper_bgcolor = "#091335 ",
                    showlegend=False)
  return fig


In [15]:
#Counter will not work woth nan vaues so removing the nan values.
data.dropna(axis = 0, inplace = True)

#Function to find the fav cuisiens of countries
def fav_cuisiens_countries(x):
    c1 = []
    c2 = []
    
    for i in data[data["Country Name"] == x]["Cuisines"].values:
        c1.append(i.split(","))
    for i in c1:
        for j in i:
            c2.append(j)
    l = dict(Counter(c2))
    from heapq import nlargest
    res = nlargest(1, l, key = l.get)
    return res
#top revanued city
def max_perticipant(x):
    max_city = data[data["Country Name"] == x].groupby("City").sum().sort_values("Average cost  in INR", ascending = False).index[0]
    return max_city
#least revanued city
def min_perticipant(x):
    min_city = data[data["Country Name"] == x].groupby("City").sum().sort_values("Average cost  in INR", ascending = False).index[-1]
    return min_city
#top revaneu in a single city
def amt_max_cont(x):
    amt_max = data[data["Country Name"] == x].groupby("City").sum().sort_values("Average cost  in INR", ascending = False).iloc[0, 6]
    return amt_max
#leadt revanue in a single city
def amt_min_cont(x):
    amt_min = data[data["Country Name"] == x].groupby("City").sum().sort_values("Average cost  in INR", ascending = False).iloc[-1, 6]
    return amt_min

def iso_code(x):    
    import pycountry
    user_input = x
    mapping = {country.name: country.alpha_3 for country in pycountry.countries}
    return mapping.get(user_input)

In [16]:
Country_data = data.groupby("Country Name").mean().iloc[:, [6,7,8]]
Country_data["Total amount of money spent in a country"] = data.groupby("Country Name").sum().iloc[:, 6]
Country_data["Sum of voted in a country"] = data.groupby("Country Name").sum().iloc[:, 9]
Country_data["Country Name"] = Country_data.index
Country_data["S.No"] = [i for i in range(1, Country_data.shape[0]+1)]
Country_data.set_index("S.No", inplace = True)
Country_data["Maximum contributed city"] = Country_data["Country Name"].apply(lambda x:max_perticipant(x))
Country_data["Maximum contributed amount"] = Country_data["Country Name"].apply(lambda x:amt_max_cont(x))
Country_data["Minimum contributed city"] = Country_data["Country Name"].apply(lambda x:min_perticipant(x))
Country_data["Minimum contributed amount"] = Country_data["Country Name"].apply(lambda x:amt_min_cont(x))
Country_data["Favourite cuisien"] = Country_data["Country Name"].apply(lambda x:fav_cuisiens_countries(x)[0])
Country_data["ISO_alpha"] = Country_data["Country Name"].apply(lambda x:iso_code(x))
Country_data.iloc[-3, -1] = "UAE"
Country_data.iloc[6, -1] = "PHL"
online_order_by_country = {}
dine_in_by_country = {}
for i in zip(data[data["Mode of Order"] == "Dine In"].groupby("Country Name").sum().iloc[:, 6].index, data[data["Mode of Order"] == "Dine In"].groupby("Country Name").sum().iloc[:, 6].tolist()):
  dine_in_by_country[i[0]] = round(i[1], 2)
for i in zip(data[data["Mode of Order"] == "Online Delivery"].groupby("Country Name").sum().iloc[:, 6].index, data[data["Mode of Order"] == "Online Delivery"].groupby("Country Name").sum().iloc[:, 6].tolist()):
  online_order_by_country[i[0]] = round(i[1], 2)

Country_data["Revanue by Online delivery"] = Country_data["Country Name"].map(online_order_by_country)
Country_data["Revanue by Dine In"] = Country_data["Country Name"].map(dine_in_by_country)
#Since dine in data of some countries were not available it is filled zero for further manipulation
Country_data.fillna(0, inplace = True)

In [17]:
#Class of functiond for creating the content for the table
class Country_details:
  def __init__(self, c_name):
    self.c_name = c_name

  def fav_cuis_country(self):
    cuis = Country_data[Country_data["Country Name"] == self.c_name].iloc[0, -4]
    return cuis

  def t_contribution(self):
    t_cont = Country_data[Country_data["Country Name"] == self.c_name].iloc[0, 3]
    return "₹"+str(round(t_cont, 2))

  def avg_spend(self):
    avg_spnd = Country_data[Country_data["Country Name"] == self.c_name].iloc[0, 0]
    return "₹"+str(round(avg_spnd, 2))

  def max_city(self):
    max_city = Country_data[Country_data["Country Name"] == self.c_name].iloc[0, 6]
    return max_city
  
  def min_city(self):
    min_city = Country_data[Country_data["Country Name"] == self.c_name].iloc[0, 8]
    return min_city

#*********************************************************************************************************************************************************************************
#Function for creating table for the country wise analysis
def tabulate_country_data(x):
    country_analysis = Country_details(x)
    o1 = country_analysis.fav_cuis_country()
    o2 = country_analysis.t_contribution()
    o3 = country_analysis.avg_spend()
    o4 = country_analysis.max_city()
    o5 = country_analysis.min_city()
    ctry_data=  pd.DataFrame({"Favourite Cuisine" : o1,
                              "Total contribution" : o2,
                              "Average spending ability of people" : o3,
                              "Maximum contributed city" : o4,
                              "Minimum contributed city" : o5}, index = [1])
    fig = go.Figure(data=[go.Table(
        columnorder = [1,2, 3, 4, 5],
        columnwidth = [100,100, 150, 150, 150],
        header = dict(
            values = [['<b>'+i+'<b>'] for i in ctry_data.columns],
            line_color='red',
            fill_color='black',
            align=['center'],
            font=dict(color='red', size=15),
            height=40
      ),
      cells=dict(
        values=[ctry_data["Favourite Cuisine"],
                ctry_data["Total contribution"], 
                ctry_data["Average spending ability of people"], 
                ctry_data["Maximum contributed city"], 
                ctry_data["Minimum contributed city"]
               ],
        line_color='red',
        fill=dict(color=['white', 'white']),
        align=['center'],
        font_size=15,
        height=30)
        )
    ])
    fig.update_layout(width=1500, height=290, paper_bgcolor = "#091335", title = "Tabular representation of country")
    
    return fig

#***************************************************************************************************************************************************************************
#Function for creating bar and pie chart of country wise analysis
def country_plot(c_name):
  fig = make_subplots(
      rows=1, cols=2,
      subplot_titles=(f'Spending habit of people in the country {c_name} in INR',
                      'Dine IN vs Online delivery'),
      specs=[[{"type": "xy"}, {"type": "domain"}]],         
  )

  fig.add_trace(
      go.Box(x = data[data["Country Name"] == c_name]["Average cost  in INR"], showlegend=False), row = 1, col = 1)
  
  fig.add_trace(go.Pie(values = data[data["Country Name"] == c_name]["Mode of Order"].value_counts().tolist(),
                        labels = data[data["Country Name"] == c_name]["Mode of Order"].value_counts().index,
                        textinfo="label+percent",
                        showlegend=False),
                  row=1, col=2)

  fig.update_layout(height=500,
                    width = 1500,
                    template = "plotly_dark",
                    plot_bgcolor = "#A3ACAC",
                    paper_bgcolor = "#091335",
                    showlegend=False)
  return fig


In [18]:
def choropleth_map(feature):
  money = ["Average cost  in INR", "Total amount of money spent in a country", "Revanue by Online delivery", "Revanue by Dine In"]
  if feature in money:
    ct = '₹'
  elif feature not in money:
    ct = ""

  fig = go.Figure(data=go.Choropleth(
      locations = Country_data['ISO_alpha'],
      z = Country_data[feature],
      text = Country_data['Country Name'],
      autocolorscale=True,
      reversescale=True,
      marker_line_color='black',
      marker_line_width=0.5,
      colorbar_tickprefix = ct,
      colorbar_title = feature
  ))

  fig.update_layout(
      autosize=False,
      margin = dict(
                l=2,
                r=0,
                b=0,
                t=0,
                pad=4,
                autoexpand=True
            ),height = 500, width = 1500,
            paper_bgcolor = "#091335",
      geo=dict(
          showframe=False,
          showcoastlines=False,
          projection_type='equirectangular',
          showocean=True, oceancolor="LightBlue"
      ),
      annotations = [dict(
          x=0.1,
          y=0.1,
          xref='paper',
          yref='paper',
          showarrow = False
      )]
  )
  return fig

def scatter_map(feature):
  money = ["Average cost  in INR", "Total amount of money spent in a country", "Revanue by Online delivery", "Revanue by Dine In"]
  if feature in money:
    ct = '₹'
  elif feature not in money:
    ct = ""
  fig = px.scatter_geo(Country_data, locations="ISO_alpha",
                      color=feature, # lifeExp is a column of gapminder
                      hover_name="Country Name", # column to add to hover information
                      color_continuous_scale=px.colors.sequential.Plasma,
                      size = feature,
                      hover_data = ["Favourite cuisien",
                                    "Maximum contributed city", 
                                    "Maximum contributed amount",
                                    "Minimum contributed city",
                                    "Minimum contributed amount",
                                    ])
  fig.update_layout(
      autosize=False,
      margin = dict(
                l=2,
                r=0,
                b=0,
                t=0,
                pad=4,
                autoexpand=True
            ),height = 500, width = 1500,
            paper_bgcolor = "#091335",
      geo=dict(
          showframe=False,
          showcoastlines=False,
          projection_type='equirectangular',
          showocean=True, 
          oceancolor="LightBlue"
      ))
  return fig




In [22]:
#from dash import Dash
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
#
load_figure_template(["sketchy", "cyborg", "minty"])
#app = Dash(__name__)

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.VAPOR])

fig6 = px.icicle(overall_indian_data, path=[px.Constant("all"), 'Mode of Order','City', "State"],
                values='Total cost in log', color='State', hover_name = "Total Cost in INR", hover_data = ["No of Orders", "Total Cost in INR"])
fig6.update_layout(margin = dict(t=50, l=25, r=25, b=25), paper_bgcolor = "#091335")

app.layout = html.Div([
   html.H1("Zomato Conutry wise sales analysis", style = {"color" : "red", "textAlign"  : "center"}),
   html.Br(),
   html.H4("Select the Country"),
   html.Div([
       dcc.Dropdown(
           id = "countryname",
           options = [{"label" : x, "value" : x} for x in data["Country Name"].unique()],
           placeholder = "India",
           clearable=True   
       )
   ]),
   html.Br(),
   dcc.Graph(
       id = "fig3"
   ),
   #html.Br(),
   dcc.Graph(
       id = "fig4"
       ),
     html.H1("Zomato data Indian Citywise analysis", style = {"color" : "red", "textAlign"  : "center"}),
     html.Br(),
     html.H4("Select the city"),
   html.Div([
       dcc.Dropdown(
           id = "cityname",
           options = [{"label" : x, "value" : x} for x in Indian_data["City"].unique()],
           value = "Chennai",
           clearable=True   
       )
   ]),
   html.Br(),
   dcc.Graph(
       id = "fig1"
   ),
   html.Br(),
   dcc.Graph(
       id = "fig2"
       ),
    html.Br(),
    html.H1("Revanue of the Indian Cities with respect to their delivery modes", style = {"color" : "red", "textAlign" : "center", "size" : 15}),
    dcc.Graph(
        id = "mode_of_delivery_analysis",
        figure = fig6

  ),
  html.H1("Zomato worldwide sales analysis", style = {"color" : "red", "textAlign"  : "center"}),
  html.H4("Select the featur"),
  html.Div([
      dcc.Dropdown(
          id = "featurename",
          options = [{"label" : "Country wise spending nature" ,"value" :  "Average cost  in INR"},
                     {"label" : "Gross revaneu by country" , "value" : "Total amount of money spent in a country"},
                     {"label" : "Revaneu by Online delivery" , "value" : "Revanue by Online delivery"},
                     {"label" : "Revaneu by Dine In" , "value" : "Revanue by Dine In"},
                     {"label" : "Average Rating by the customers by country" , "value" : "Aggregate rating"},
                     {"label" : "Prefered price range by country" , "value" : "Price range"}],
                   value = "Gross revaneu by country",
                   clearable=True   
       )
   ]),
   html.Br(),
   html.H4("Select the map type"),
   dcc.RadioItems(
       id = "map_selection",
       options = ['Choropleth Visuvalization','Scattered Visuvalization'],
       value =  'Choropleth Visuvalization'),
  #html.Br(),
  html.H2([], 
          style = {"Color" : "red", "textAlign" : "center", "size" : 15},
          id = "map_name"),
  html.Br(),

   dcc.Graph(
       id = "fig5"
   ),
   html.Br(),
   html.H6("This dashboard is only for educational purpose and developed by G.TAMIZHARASAN", style = {"textAlign" : "center"})

   
])
 
 
@app.callback(
    Output(component_id='fig3', component_property='figure'),
    Input(component_id='countryname', component_property='value'),
    prevent_initial_call=True
)
def update_tabel_country(input_value):
  return tabulate_country_data(input_value)

@app.callback(
    Output(component_id='fig4', component_property='figure'),
    Input(component_id='countryname', component_property='value'),
    prevent_initial_call=True
)
def update_charts_country(input_value):
  return country_plot(input_value)

@app.callback(
    Output(component_id='fig1', component_property='figure'),
    Input(component_id='cityname', component_property='value'),
    prevent_initial_call=True
)
def update_tabel_city(input_value):
  return tabulate_city_data(input_value)

@app.callback(
    Output(component_id='fig2', component_property='figure'),
    Input(component_id='cityname', component_property='value'),
    prevent_initial_call=True
)
def update_chart_india(input_value):
  return city_plot(input_value)

@app.callback(
    Output(component_id='map_name', component_property='children'),
    Input(component_id='featurename', component_property='value'),
)
def title(name): 
  return name.upper()

 
@app.callback(
    Output(component_id='fig5', component_property='figure'),
    Input(component_id='featurename', component_property='value'),
    Input(component_id='map_selection', component_property='value'),
    prevent_initial_call=True
)
def update_map(comp, map_selection):
  if map_selection == "Choropleth Visuvalization":
    return choropleth_map(comp)
  elif map_selection == "Scattered Visuvalization":
    return scatter_map(comp)




if __name__ == '__main__':
   app.run_server(mode = "external", port = 8600)

Dash app running on:


<IPython.core.display.Javascript object>